In [1]:
# a functional api equivalent to a Sequential model
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# a Sequential model
seq_model=Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# its functional equivalent
input_tensor=Input(shape=(64,))
x=layers.Dense(32, activation='relu')(input_tensor)
x=layers.Dense(32, activation='relu')(x)
output_tensor=layers.Dense(10, activation='softmax')(x)

#the Model class turns an input tensor and output tensor into a model
model=Model(input_tensor, output_tensor)

model.summary()

D:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
# compile and train the functional model
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

import numpy as np
x_train=np.random.random((1000,64))
y_train=np.random.random((1000,10))

model.fit(x_train,y_train,epochs=10,batch_size=128)
model.evaluate(x_train,y_train)

Epoch 1/10
1000/1000 [==============================] - 0s 198us/step - loss: 11.5340
Epoch 2/10
1000/1000 [==============================] - 0s 25us/step - loss: 11.5268
Epoch 3/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.5251
Epoch 4/10
1000/1000 [==============================] - 0s 33us/step - loss: 11.5231
Epoch 5/10
1000/1000 [==============================] - 0s 27us/step - loss: 11.5204
Epoch 6/10
1000/1000 [==============================] - 0s 25us/step - loss: 11.5192
Epoch 7/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.5179
Epoch 8/10
1000/1000 [==============================] - 0s 34us/step - loss: 11.5165
Epoch 9/10
1000/1000 [==============================] - 0s 34us/step - loss: 11.5154
Epoch 10/10
1000/1000 [==============================] - 0s 68us/step


11.51127272796631

In [2]:
# Multiple input model: a question-answering model
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size=10000
question_vocabulary_size=10000
answer_vocabulary_size=500

# the text input is a variable-length sequence of integers
text_input=Input(shape=(None, ),dtype='int32', name='text')
# embed the text input into a sequence of vectors of size 64
embedded_text=layers.Embedding(64, text_vocabulary_size)(text_input)
# decode in a single vector via a LTSM
encoded_text=layers.LSTM(32)(embedded_text)

# same process for the question
question_input=Input(shape=(None, ),dtype='int32', name='question')
embedded_question=layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question=layers.LSTM(16)(embedded_question)

# concatenate the encoded question and encoded text
concatenated=layers.concatenate([encoded_text, encoded_question], axis=-1)

# add a softmax classifier on top
answer=layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# specify the two inputs and the output at model instantiation
model=Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

D:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# feed data to a multi-input model
import numpy as np

num_samples=10000
max_length=10

# generate some dummy Numpy datamax_length
text=np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question=np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
# answers are one-hot encoded, not integers
answers=np.random.randint(0,1,size=(num_samples, answer_vocabulary_size))

# fit using a list of input
model.fit([text, question], answers, epochs=10, batch_size=128)

# fit using a dictionary pf inputs (only if inputs were named)
model.fit({'text':text, 'question':question}, answers, epochs=10, batch_size=128)

In [10]:
# Multi-output model: a three-output model
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size=50000
num_income_groups=10

posts_input=Input(shape=(None, ), dtype='int32', name='posts')
embedded_posts=layers.Embedding(256, vocabulary_size)(posts_input)
x=layers.Conv1D(128,5,activation='relu')(embedded_posts)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.GlobalMaxPooling1D()(x)
x=layers.Dense(128,activation='relu')(x)

# give names to the output layers
age_prediction=layers.Dense(1, name='age')(x)
income_prediction=layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction=layers.Dense(1, activation='sigmoid', name='gender')(x)

model=Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [ ]:
# compilation options of a multi-output model: multiple losses
model.compile(optimizer='rmsprop', loss=['mae', 'categorical_crossentropy', 'binary_crossentropy'])
# equivalent (only if the output layers are given names)
model.compile(optimizer='rmsprop', loss={'age':'mae', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'})

# loss weighting
model.compile(optimizer='rmsprop', loss=['mae', 'categorical_crossentropy', 'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])
# equivalent
model.compile(optimizer='rmsprop', loss={'age':'mae', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'},
             loss_weights={'age':0.25, 'income':1., 'gender':10.})

In [ ]:
# feed data to a multi-output model
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)
# equivalent
model.fit(posts, {'age':age_targets, 'income':income_targets, 'gender':gender_targets}, epochs=10, batch_size=64)

In [ ]:
# Inception module with the functional api: Inception V3
from keras import layers

# assume the existence of a 4D input tensor 'x'

# every branch has the same stride value (2), to keep all branch outputs the same size
branch_a=layers.Conv2D(128, 1, activation='relu', strides=2)(x)

# the striding occurs in the spatial convolution layer
branch_b=layers.Conv2D(128, 1, activation='relu')(x)
branch_b=layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

# the striding occurs in the average pooling layer
branch_c=layers.AveragePooling2D(3, activation='relu', strides=2)(x)
branch_c=layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d=layers.Conv2D(128, 1, activation='relu')(x)
branch_d=layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d=layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

# concatenate the branch outputs to obtain the module output
output=layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

In [ ]:
# Residual connection
from keras import layers

# assume the existence of a 4D input tensor 'x'
x=...

# when feature map sizes are the same
y=layers.Conv2D(128, 3, activation='relu')(x)
y=layers.Conv2D(128, 3, activation='relu')(y)
y=layers.Conv2D(128, 3, activation='relu')(y)

# add the original 'x' back to the output features
y=layers.MaxPooling2D(2, strides=2)(y)

In [ ]:
# when feature map sizes are different
y=layers.Conv2D(128, 3, activation='relu')(x)
y=layers.Conv2D(128, 3, activation='relu')(y)
y=layers.MaxPooling2D(2, strides=2)(y)

# use a 1x1 convolution to linearly downsample the original 'x' tensor to be the same shape as 'y'
residual=layers.Conv2D(1, strides=2)(x)

# add the residual tensor back to the output features
y=layers.add([y,residual])

In [ ]:
# Layer weight sharing: a shared LSTM model
from keras import layers
from keras import Input
from keras.models import Model

# instantiate a single LSTM layer, once
lstm=layers.LSTM(32)

# build the left branch of the model
left_input=Input(shape=(None, 128))
left_output=lstm(left_input)

# build the right branch of the model
right_input=Input(shape=(None, 128))
# when calling an existing layer instance, we are reusing its weights
right_output=lstm(right_input)

# build the classifier on top
merged=layers.concatenate([left_output, right_output], axis=-1)
predictions=layers.Dense(1, activation='sigmoid')(merged)

# instantiate and train the model
model=Model([left_input, right_input], predictions)
# when training such a model, the weights of the lstm layer are updated based on both inputs
model.fit([left_data, right_data], targets)

In [ ]:
# Models as layers: a shared vision model
from keras import layers
from keras import applications
from keras import Input

# the base image processing model will be the Xception network
xception_base=applications.Xception(weights=None, include_top=False)

# the inputs are 250x250 RGB images
left_input=Input(shape=(250,250,3))
right_input=Input(shape=(250,250,3))

# call the same vision model twice, and reuse its weights
left_features=xception_base(left_input)
right_features=xception_base(right_input)

# the merged features contain information from both the right visual feed and the left visual feed
merged_features=layers.concatenate([left_features, right_features], axis=-1)